In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import signal
%matplotlib inline

#Import the open/close time data.

In [42]:
df = pd.read_csv(filepath_or_buffer = 'reference_data/all_results.csv', index_col = False)
print df[:5]

   year  month  day  hour  min     sec  close_hour  cmin  csec  open_hour  \
0  2014      7   18    17   20  25.278          16    39     3         16   
1  2014      7   18    18   24   4.878          17    42    45         17   
2  2014      7   18    19   27  32.778          18    46    18         18   
3  2014      7   18    20   31   6.978          19    49    44         19   
4  2014      7   18    21   34  51.978          20    53    25         20   

   omin  osec    camb     css      flux  
0    38    33  41.183  42.369  0.016510  
1    42    15  49.626  37.207 -0.172850  
2    45    48  26.638  46.415  0.275260  
3    49    14  34.020  18.391 -0.217530  
4    52    55  30.356  30.103 -0.003522  


####Convert the time columns to python time stamps.

In [44]:
df['sec'] = df['sec'].map(lambda x: 0 if x == 60 else x)
def f(x): return round(x)
df['sec'] = map(f, df['sec'])
df['closing'] = pd.to_datetime(df.year.astype(str) + '-' + df.month.astype(str) + '-' + df.day.astype(str) + ' ' + df.open_hour.astype(str) + ':' + df['omin'].astype(str) + ':' + df.osec.astype(str))
df['closed'] = pd.to_datetime(df.year.astype(str) + '-' + df.month.astype(str) + '-' + df.day.astype(str) + ' ' + df.close_hour.astype(str) + ':' + df['cmin'].astype(str) + ':' + df.csec.astype(str))
df['end_of_run'] = pd.to_datetime(df.year.astype(str) + '-' + df.month.astype(str) + '-' + df.day.astype(str) + ' ' + df.hour.astype(str) + ':' + df['min'].astype(str) + ':' + df.sec.astype(str))
df = df.set_index(pd.DatetimeIndex(df['end_of_run']))

In [45]:
df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9,10,11,12,13]], axis=1, inplace=True)

In [46]:
df.head()

,flux,closing,closed,end_of_run
2014-07-18 17:20:25,0.016510,2014-07-18 16:38:33,2014-07-18 16:39:03,2014-07-18 17:20:25
2014-07-18 18:24:05,-0.172850,2014-07-18 17:42:15,2014-07-18 17:42:45,2014-07-18 18:24:05
2014-07-18 19:27:33,0.275260,2014-07-18 18:45:48,2014-07-18 18:46:18,2014-07-18 19:27:33
2014-07-18 20:31:07,-0.217530,2014-07-18 19:49:14,2014-07-18 19:49:44,2014-07-18 20:31:07
2014-07-18 21:34:52,-0.003522,2014-07-18 20:52:55,2014-07-18 20:53:25,2014-07-18 21:34:52


##Import the reference data.

In [47]:
df_R = pd.read_csv(filepath_or_buffer = 'reference_data/chamber_2b_data/all_ref_data.csv', parse_dates = [0], index_col = 0)
print df_R[:5]

                      o3_ppb
date                        
2014-07-25 12:06:00       65
2014-07-25 12:07:00       67
2014-07-25 12:08:00       68
2014-07-25 12:09:00       66
2014-07-25 12:10:00       64


In [48]:
df['start'] = df.closing - pd.Timedelta(minutes=10)
df['stop'] = df.closed + pd.Timedelta(minutes=10)

In [49]:
df.irow(148)

flux                     -0.10543
closing       2014-07-25 15:42:31
closed        2014-07-25 15:43:01
end_of_run    2014-07-25 16:24:22
start         2014-07-25 15:32:31
stop          2014-07-25 15:53:01
Name: 2014-07-25 16:24:22, dtype: object

In [50]:
df_new = pd.DataFrame({'o3_ppb':[]})
date_list = []
o3_list = []
for row_index, row in df.iterrows():
    for row_index_ref, row_ref in df_R.iterrows():
        if row_index_ref >= (row.closed + pd.Timedelta(minutes=4)) and row_index_ref <= (row.closed + pd.Timedelta(minutes=7)):
            date_list.append(row_index_ref)
            o3_list.append(int(row_ref.values))

In [51]:
df_new.o3_ppb = o3_list
df_new.index = date_list

In [52]:
df_new[:50]

,o3_ppb
2014-07-25 23:25:00,38
2014-07-25 23:26:00,37
2014-07-25 23:27:00,39
2014-07-25 15:48:00,40
2014-07-25 15:49:00,42
2014-07-25 15:50:00,43
2014-07-25 16:51:00,39
2014-07-25 16:52:00,37
2014-07-25 16:53:00,36
2014-07-25 17:55:00,40


In [53]:
df_new.to_csv(path_or_buf = 'reference_data/closed_only.csv')